# GMAC Optimization: Grid search of GMAC parameters

The GMAC algorithm has the following parameters required to map the raw acceleration
data to the arm-use space. These parameters are:
1. Moving average window size for pitch estimation. $\left( n_{p} \right)$
2. Butteworth highpass filter cut-off for accl. mag. estimation. $\left( f_{c} \right)$
3. Butteworth highpass filter order for accl. mag. estimation. $\left( n_{c} \right)$
4. Acceleration deadband threshold. $\left( a_{th0} \right)$
5. Moving average window for acceleration magnitude estimation. $\left( n_{am} \right)$
6. Pitch angle hysteresis lower threshold. $\left( \theta_{min} \right)$
7. Pitch angle hysteresis upper threshold. $\left( \theta_{max} \right)$
8. Acceleration magnitude hysteresis lower threshold. $\left( a_{min} \right)$
9.  Acceleration magnitude hysteresis upper threshold. $\left( a_{max} \right)$

## GMAC Algorithm

The GMAC algorithm is a combination of the gross movement (GM) algorithm and the 
activty counting (AC) algorithm. The GMAC algorithm, using only the accelerometer, 
tries to: (1) estimate the information used by GM and AC, forearm pitch angle and 
activity counts, respectively; and (2) applies thresholds on the estimated pitch 
and activity counting to estimate of the arm-use.

### Pitch Estimation
Pitch estimation is using the accelerometer data, and requies one to know the axis 
of accelerometer that is aligned with the forearm. The pitch is then estimated by 
estimating the arc-cosine of component of acceleration along this direction normalised 
by the magnitude of the acceleration due to gravity.

### Activity Counting Estimation
The activity counts is likely to be correlated to the magnitude of the highpass 
filtered acceleration.

### Applying thresholds
The pitch and activity counts are then thresholded to estimate the arm-use. 
Instead of applying a simple threshold algorithm that can be very sensitive to 
noise whenn the variable is close to the threshold, a thresholding procedure with 
hysteresis algorithm is used for both the pitch and the acceleration magnitude 
variables.

### Parameter search ranges
$F_s$ is the sampling frequency of the accelerometer data.
1. $n_{p} \in \left\{1, F_s/2, F_s, 2F_s, 5F_s\right\}$
2. $f_{c} \in [0.01, 0.1, 1]Hz \subset \mathbb{R}$
3. $n_{c} \in \left\{ 2, 4, 8\right\}$
4. $n_{am} \in \left\{1, F_s/2, F_s, 2F_s, 5F_s\right\}$
5. $a_{th0} \in \left\{ 0, 0.068, 0.68\right\}$
6. $\theta_{min}, \theta_{max} \in \left\{ 0, 20, \ldots +180 \right\}\deg$
7. $a_{min}, a_{max} \in \left\{0, 5, 10, 50, 100\right\}$

### Getting the data

If you do not already have the data to run this this notebook, you need to
download it from here.



### Standards modules

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import scipy
from scipy import signal
import pathlib
import itertools
import glob

In [3]:
import seaborn as sns

### Custom Modules

In [4]:
sys.path.append("../scripts")

import classification_algorithms as ca
import reduced_models as rm
import task_analysis as ta

import misc

## Define notebook level constants

In [5]:
# Sampling rate for the data is 50ms
dT = 0.02
Fs = int(1 / dT)

## Read the Controls and Patients Data

In [121]:
# Read healthy and control data
left, right = misc.read_data(subject_type='control')
aff, unaff = misc.read_data(subject_type='patient')

# Assign segments for each subject
left = pd.concat([misc.assign_segments(left[left.subject == subj], dur_th=1, dT=dT)
                  for subj in left.subject.unique()], axis=0)
right = pd.concat([misc.assign_segments(right[right.subject == subj], dur_th=1, dT=dT)
                   for subj in right.subject.unique()])
aff = pd.concat([misc.assign_segments(aff[aff.subject == subj], dur_th=1, dT=dT)
                 for subj in aff.subject.unique()])
unaff = pd.concat([misc.assign_segments(unaff[unaff.subject == subj], dur_th=1, dT=dT)
                   for subj in unaff.subject.unique()])

# All limbs data ddf
datadf = {
    "left": left,
    "right": right,
    "aff": aff,
    "unaff": unaff
}

### Functions implementing the GMAC algorithm

In [6]:
def estimate_pitch(accl_farm: np.array, nwin: int) -> np.array:
    """
    Estimates the pitch angle of the forearm from the accelerometer data.
    """
    # Moving averaging using the causal filter
    acclf = signal.lfilter(np.ones(nwin) / nwin, 1, accl_farm) if nwin > 1 else accl_farm
    acclf[acclf < -1] = -1
    acclf[acclf > 1] = 1        
    return -np.rad2deg(np.arccos(acclf)) + 90


def estimate_accl_mag(accl: np.array, fs: float, fc: float, nc: int,
                      deadband_th: float, n_am: int) -> np.array:
    """
    Compute the magnitude of the accelerometer signal.
    """
    # Highpass filter the acceleration data.
    sos = signal.butter(nc, fc, btype='highpass', fs=fs, output='sos')
    accl_filt = np.array([signal.sosfilt(sos, accl[:, 0]),
                          signal.sosfilt(sos, accl[:, 1]),
                          signal.sosfilt(sos, accl[:, 2])]).T
    
    # Zero load acceleration components.
    accl_filt[np.abs(accl_filt) < deadband_th] = 0
    
    # Acceleration magnitude    
    amag = np.linalg.norm(accl_filt, axis=1)
    
    # Moving average filter
    _input = np.append(np.ones(n_am - 1) * amag[0], amag)
    _impresp = np.ones(n_am) / n_am
    return np.convolve(_input, _impresp, mode='valid')

In [13]:
def estimation_param_combinations(param_ranges: dict) -> tuple:
    """
    Generate the list of parameter combinations for estimating the pitch and 
    acceleration magnitude.
    """
    # Pitch and accl. mag. estimation parameters
    for _np in param_ranges["np"]:
        for _fc in param_ranges["fc"]:
            for _nc in param_ranges["nc"]:
                for _ath in param_ranges["deadband_th"]:
                    for _nam in param_ranges["nam"]:
                        yield (_np, _fc, _nc, _ath, _nam)


# Generate all possible combinations of parameters.
def generate_param_combinations(param_ranges: dict) -> dict:
    """
    Generate all possible combinations of parameters.
    """
    for _estparams in estimation_param_combinations(param_ranges):
        for i1, _pmax in enumerate(param_ranges["p_th"]):
            for _pmin in param_ranges["p_th"][:i1+1]:
                for i2, _amax in enumerate(param_ranges["am_th"]):
                    for _amin in param_ranges["am_th"][:i2+1]:
                        yield {
                            "np": int(_estparams[0]),
                            "fc": _estparams[1],
                            "nc": int(_estparams[2]),
                            "deadband_th": _estparams[3],
                            "nam": int(_estparams[4]),
                            "p_thmin": _pmin,
                            "p_thmax": _pmax,
                            "am_thmin": _amin,
                            "am_thmax": _amax
                        }

In [8]:
def apply_threshold_with_hysteresis(data: np.array, thmin: float, thmax: float,
                                    out0: float=0):
    """
    Applies threshold to the given data using the thmin and thmax thresholds.
    """
    _out = np.zeros(len(data))
    _out[0] = out0
    for i in range(1, len(_out)):
        if _out[i - 1] == 0:
            _out[i] = 1 if data[i] >= thmax else 0
        else:
            _out[i] = 0 if data[i] < thmin else 1
    return _out

In [9]:
def estimate_gmac(accl: np.array, accl_farm_inx: int, Fs: float, params: dict) -> np.array:
    """
    Estimate GMAC for the given acceleration data and parameters.
    """
    # Estimate pitch and acceleration magnitude
    pitch = estimate_pitch(accl[:, accl_farm_inx], params["np"])
    accl_mag = estimate_accl_mag(accl, Fs, fc=params["fc"], nc=params["nc"],
                                 deadband_th=params["deadband_th"],
                                 n_am=params["nam"])
    
    # Compute GMAC
    _pout = apply_threshold_with_hysteresis(pitch, params["p_thmin"], params["p_thmax"])
    _amout = apply_threshold_with_hysteresis(accl_mag, params["am_thmin"], params["am_thmax"])
    return _pout * _amout

In [10]:
def compute_confusion_matrix(actual: np.array, estimated: np.array) -> np.array:
    """
    Computes the components of the confusion matrix.
    """
    actual = np.array(actual, dtype=int)
    estimated = np.array(estimated, dtype=int)
    return {
        "TN": np.sum((actual + estimated) == 0), # TN
        "FP": np.sum((1 - actual + estimated) == 2), # FP
        "FN": np.sum((actual + 1 - estimated) == 2), # FN
        "TP": np.sum((actual + estimated) == 2), # TP
    }

#### Parameter ranges for the grid search

In [14]:
outdir = "../data/output"
limbkey = "left"
# Make directory if it does not exist
pathlib.Path(outdir, limbkey).mkdir(parents=True, exist_ok=True)

# GMAC Parameter ranges
gmac_param_ranges = {
    "np": [1, Fs, 5*Fs],
    "fc": [0.01, 0.1, 1],
    "nc": [2, 4],
    "deadband_th": [0, 0.068, 0.68],
    "nam": [1, Fs, 5*Fs],
    "p_th": np.arange(-90, 90, 30),
    "am_th": [0, 1, 5, 10]
}

# Number of all possible combinations
Ncombs = len(list(generate_param_combinations(gmac_param_ranges)))

In [242]:
# All parameter combinations.
param_combs = list(generate_param_combinations(gmac_param_ranges))

# Which limb?
limbdf = datadf[limbkey]
subjects = limbdf.subject.unique()

# All param combinations.
analysisdf = pd.DataFrame(columns=["subject", "pcinx",
                                   "np", "fc", "nc", "deadband_th", "nam",
                                   "p_thmin", "p_thmax",
                                   "am_thmin", "am_thmax",
                                   "TN", "FP", "FN", "TP"])
nrows_write = 1000
for i1, _pc in enumerate(generate_param_combinations(gmac_param_ranges)):
    # All subjects
    for i2, _subj in enumerate(subjects):
        _sinx= limbdf.subject == _subj
        # All segments
        segs = limbdf[_sinx].segment.unique()
        _gmacs = []
        for i3, _seg in enumerate(segs):
            sys.stdout.write(f"\rCombination: {i1+1:8d}/{Ncombs:6d}; {i2+1:3d}/{len(subjects):3d}; {i3+1:2d}/{len(segs):2d}")
            sys.stdout.flush()
            _ainx = _sinx & (limbdf.segment == _seg)
            # Estimate GMAC
            _accl = limbdf.loc[_ainx, ['ax', 'ay', 'az']].values
            _gmacs.append(estimate_gmac(_accl, 0, Fs, _pc))
        
        # Estimate confusion matrix components
        _perf = compute_confusion_matrix(actual=limbdf.loc[_sinx, "gnd"].values,
                                         estimated=np.hstack(_gmacs))
        # Update analysis DF
        _rowdf = pd.DataFrame({"subject": _subj, "pcinx": i1} | _pc | _perf, index=[0])
        analysisdf = pd.concat([analysisdf,_rowdf],
                               axis=0, ignore_index=True)
    # Save data regularly
    if i1 > 0 and i1 % nrows_write == 0:
        analysisdf.to_csv(pathlib.Path(outdir, limbkey, f"analysisdf-{(i1 // nrows_write) - 1}.csv"), index=False)
        analysisdf = pd.DataFrame(columns=["subject", "pcinx",
                                           "np", "fc", "nc", "deadband_th", "nam",
                                           "p_thmin", "p_thmax",
                                           "am_thmin", "am_thmax",
                                           "TN", "FP", "FN", "TP"])
# Save whatever is left
analysisdf.to_csv(pathlib.Path(outdir, limbkey, f"analysisdf-{(i1 // nrows_write)}.csv"), index=False)

Combination:    34020/ 34020;  10/ 10;  2/ 2

## Find the most appropriate paramters

In [41]:
def performance(yjvals: np.array) -> dict:
    """
    Computes the performance of the given GMAC.
    """
    q5, q50, q95 = np.percentile(yjvals, q=[5, 50, 95])
    return {
        "median": q50,
        "q5": q5,
        "q95": q95,
        "s90": q95 - q5,
        "perf": np.abs(q50 - 1) + 0.5 * (q95 - q5)
    }

In [15]:
# Read all data and combine into a sigle DF
files = glob.glob(pathlib.Path(outdir, limbkey, "analysisdf*.csv").as_posix())
allanalysisdf = pd.read_csv(files[0], index_col=False)
for _f in files[1:]:
    _df = pd.read_csv(_f, index_col=False)
    allanalysisdf = pd.concat([allanalysisdf, _df], axis=0, ignore_index=True)
allanalysisdf

,subject,pcinx,np,fc,nc,deadband_th,nam,p_thmin,p_thmax,am_thmin,am_thmax,TN,FP,FN,TP
0,2,6001,1,0.10,4,0.000,50,-30,30,0,1,7049,80,15878,1255
1,3,6001,1,0.10,4,0.000,50,-30,30,0,1,4902,4125,5006,8142
2,4,6001,1,0.10,4,0.000,50,-30,30,0,1,5210,2669,6776,7069
3,5,6001,1,0.10,4,0.000,50,-30,30,0,1,4605,3466,5838,6371
4,6,6001,1,0.10,4,0.000,50,-30,30,0,1,7551,1715,10608,6654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340195,7,12000,50,0.01,2,0.068,1,-90,-30,0,0,3,9268,1,14234
340196,8,12000,50,0.01,2,0.068,1,-90,-30,0,0,2,7967,0,19449
340197,9,12000,50,0.01,2,0.068,1,-90,-30,0,0,3,21155,0,16160
340198,10,12000,50,0.01,2,0.068,1,-90,-30,0,0,3,8690,1,13573


In [16]:
# Compute different performance metrics
# Sensitivity
allanalysisdf["sensitivity"] = allanalysisdf.TP / (allanalysisdf.TP + allanalysisdf.FN)
# Specificity
allanalysisdf["specificity"] = allanalysisdf.TN / (allanalysisdf.TN + allanalysisdf.FP)
# Youden's J
allanalysisdf["youden"] = allanalysisdf.sensitivity + allanalysisdf.specificity - 1

# Save analysis DF
allanalysisdf.to_csv(pathlib.Path(outdir, limbkey, "allanalysisdf.csv"), index=False)

In [120]:
# Subjects
subjs = np.sort(allanalysisdf.subject.unique())
q = 95

# Optimization data frame
optdf = pd.DataFrame(columns=["pcinx", "median", "q95", "q5", "s90", "cost", "testval", "testsubj"])
for _nc in range(Ncombs):
    for sinx in range(len(subjs)):
        sys.stdout.write(f"\rCombination: {_nc+1:8d}/{Ncombs:6d}; {sinx+1:3d}/{len(subjs):3d}")
        sys.stdout.flush()
        testsubj = subjs[sinx]
        trainsubj = subjs[~(subjs == testsubj)]
        
        # Find the best parameter combination for the training subjects.
        _subjtraininx = allanalysisdf.subject.isin(trainsubj)
        _subjtestinx = allanalysisdf.subject == testsubj
        _inxtrain = allanalysisdf.pcinx == _nc
        _inxtest = allanalysisdf.pcinx == _nc
        
        # Find the best parameter combination for the training subjects.
        _yitrain = allanalysisdf.loc[_subjtraininx & _inxtrain, "youden"]
        _yitest = allanalysisdf.loc[_subjtestinx & _inxtest, "youden"]
        
        # Performance vairables.
        _perf = performance(_yitrain.values)
        _perfdf = pd.DataFrame({
            "pcinx": _nc,
            "median": _perf["median"],
            "q95": _perf["q95"],
            "q5": _perf["q5"],
            "s90": _perf["s90"],
            "cost": _perf["perf"],
            "testval": _yitest.values,
            "testsubj": testsubj
        })
        optdf = pd.concat([optdf, _perfdf], ignore_index=True)

# Save the optimization data frame        
optdf.to_csv(pathlib.Path(outdir, limbkey, f"optdf.csv"), index=False)

Combination:    34020/ 34020;  10/ 10

In [124]:
# Go through each validation subject, and choose the best training cost 
# parameter combination.
bestparamsdf = pd.DataFrame(columns=["subj", "lbound", "bestpinx", "bestval"])
for _subj in optdf.testsubj.unique():
    # _minval = np.min(optdf.loc[optdf.testsubj == _subj, "cost"])
    _sinx = optdf.testsubj == _subj
    _lbound = np.percentile(optdf.loc[optdf.testsubj == _subj, "cost"], .1)
    _bpins = list(np.where(optdf.loc[_sinx, "cost"] <= _lbound)[0])
    _bval = list(optdf.loc[_sinx, "testval"].values[_bpins])
    bestparamsdf = pd.concat([bestparamsdf, 
                              pd.DataFrame({"subj": [_subj] * len(_bpins),
                                            "lbound": [_lbound] * len(_bpins),
                                            "bestpinx": _bpins,
                                            "bestval": _bval})],
                             ignore_index=True)
# Save the optimization data frame
bestparamsdf.to_csv(pathlib.Path(outdir, limbkey, f"bestparamsdf.csv"), index=False)